In [192]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
# Define the seasons I need data for
seasons = [2024]

In [199]:
import pandas as pd
from typing import Callable
import SlowDB
from simulation_utils import S3_BUCKET, S3_DB_KEY

# Define update or insert function
def update_or_insert_data(table: str, columns: list[str], new_data_df: pd.DataFrame, database_function: Callable[[pd.DataFrame], pd.DataFrame] = lambda x: x) -> pd.DataFrame:
    # Call the database function on the new data
    new_data_df = database_function(new_data_df)

    with SlowDB.connect(S3_BUCKET, S3_DB_KEY) as conn:
        # Read the data from the database
        try:
            existing_df = database_function(pd.read_sql(f"SELECT * FROM {table}", conn))
        except:
            # If the table does not exist, write the new data to the database
            df = new_data_df
        else:
            # Set the index for both DataFrames
            new_data_df = new_data_df.set_index(columns)
            existing_df = existing_df.set_index(columns)

            # Find overlapping rows (by index)
            overlapping_rows = new_data_df.index.intersection(existing_df.index)

            # Drop the overlapping rows from the existing DataFrame
            existing_df.drop(overlapping_rows, axis="index", inplace=True)

            # Reset the index for both DataFrames
            new_data_df = new_data_df.reset_index()
            existing_df = existing_df.reset_index()

            # Append the new rows to the existing DataFrame
            if "season" in existing_df.columns or "season" in new_data_df.columns:
                sort_columns = ["season"]
            else:
                sort_columns = columns
            df = pd.concat([existing_df, new_data_df], ignore_index=True).sort_values(sort_columns).reset_index(drop=True)

        # Replace the data in the database
        rows_updated = df.to_sql(table, conn, if_exists="replace", index=False)

        # Print the number of rows updated
        print(f"Updated {rows_updated} rows in the {table} table")

        conn.commit()

        return df
    
# Define a function to only replace the data
def replace_data(table: str, new_data_df: pd.DataFrame) -> pd.DataFrame:
    with SlowDB.connect(S3_BUCKET, S3_DB_KEY) as conn:
        # Replace the data in the database
        rows_updated = new_data_df.to_sql(table, conn, if_exists="replace", index=False)

        # Print the number of rows updated
        print(f"Updated {rows_updated} rows in the {table} table")

        conn.commit()
    return new_data_df

In [200]:
from simulation_utils import build_data_by_year


# Get the data for the database
managers_df = pd.concat([build_data_by_year(year, False) for year in seasons])

def convert_utc_date_to_datetime(df: pd.DataFrame) -> pd.DataFrame:
    df["utc_date"] = pd.to_datetime(df["utc_date"])
    return df

# Update the database
managers_df = update_or_insert_data("football_data_season_results", ["season", "home", "away"], managers_df, convert_utc_date_to_datetime)

managers_df

Updated 1140 rows in the football_data_season_results table


,season,home,away,utc_date,status,matchday,home_score,away_score,home_outcome,away_outcome
0,2022,Liverpool FC,Chelsea FC,2023-01-21 12:30:00+00:00,FINISHED,21,0.0,0.0,1.0,1.0
1,2022,West Ham United FC,Newcastle United FC,2023-04-05 19:00:00+00:00,FINISHED,7,1.0,5.0,0.0,3.0
2,2022,Manchester City FC,Chelsea FC,2023-05-21 15:00:00+00:00,FINISHED,37,1.0,0.0,3.0,0.0
3,2022,Crystal Palace FC,Nottingham Forest FC,2023-05-28 15:30:00+00:00,FINISHED,38,1.0,1.0,1.0,1.0
4,2022,Everton FC,AFC Bournemouth,2023-05-28 15:30:00+00:00,FINISHED,38,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1135,2024,Brighton & Hove Albion FC,Southampton FC,2024-11-30 00:00:00+00:00,SCHEDULED,13,NaN,NaN,NaN,NaN
1136,2024,Brentford FC,Leicester City FC,2024-11-30 00:00:00+00:00,SCHEDULED,13,NaN,NaN,NaN,NaN
1137,2024,Southampton FC,Liverpool FC,2024-11-23 00:00:00+00:00,SCHEDULED,12,NaN,NaN,NaN,NaN
1138,2024,West Ham United FC,Arsenal FC,2024-11-30 00:00:00+00:00,SCHEDULED,13,NaN,NaN,NaN,NaN


In [201]:
# Get unique club names minus FC
from simulation_utils import get_club_value_at_season


clubs = managers_df["home"].unique()

# Get club value for each season
club_values = {}

for club in clubs:
    club_values[club] = [
        {
            "season": season,
            "value": get_club_value_at_season(club, season)
        }
        for season in seasons
    ]

# Flatten the dictionary
flattened_data = []

for club, values in club_values.items():
    for value in values:
        flattened_data.append({
            "club": club,
            "season": value["season"],
            "value": value["value"]
        })

# Convert the list of dictionaries to a DataFrame
transfers_df = pd.DataFrame(flattened_data)

transfers_df = update_or_insert_data("transfermarkt_club_values", ["club", "season"], transfers_df)

transfers_df

Updated 72 rows in the transfermarkt_club_values table


,club,season,value
0,Manchester City FC,2022,1.145950e+09
1,Crystal Palace FC,2022,3.230500e+08
2,Burnley FC,2022,1.933500e+08
3,Wolverhampton Wanderers FC,2022,4.976500e+08
4,Sheffield United FC,2022,1.185500e+08
...,...,...,...
67,Manchester City FC,2024,1.408200e+09
68,West Ham United FC,2024,4.545000e+08
69,Liverpool FC,2024,9.493000e+08
70,Newcastle United FC,2024,6.566500e+08


In [202]:
import requests
import bs4

# Get the data from the website
url = "https://en.wikipedia.org/wiki/List_of_Premier_League_managers"
response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, "html.parser")

# Get the table with classes wikitable sortable plainrowheaders jquery-tablesorter
table = soup.find("table", {"class": "wikitable sortable plainrowheaders"})

# Get the rows from the table
rows = table.find_all("tr")
rows = rows[1:]

# Get the data from the rows
data = []

for row in rows:
    cells = row.find_all(["th", "td"])

    # Get rid of all child sup elements
    for cell in cells:
        for sup in cell.find_all("sup"):
            sup.decompose()

    cells = [cell.text.strip() for cell in cells if cell.text.strip()]
    data.append(cells)

# Convert the data to a DataFrame
managers_df = pd.DataFrame(data, columns=["manager", "club", "start", "end", "duration_days", "years"])

# Add columns for keys
managers_df["incumbent"] = managers_df["manager"].apply(lambda x: True if str(x).endswith(" †") else False)
managers_df["caretaker"] = managers_df["manager"].apply(lambda x: True if str(x).endswith(" ‡") else False)
managers_df["incumbent_not_in_league"] = managers_df["manager"].apply(lambda x: True if str(x).endswith(" §") else False)

# Remove the symbols from the manager names
managers_df["manager"] = managers_df["manager"].str.replace(" †", "")
managers_df["manager"] = managers_df["manager"].str.replace(" ‡", "")
managers_df["manager"] = managers_df["manager"].str.replace(" §", "")

# Function to convert all start and end dates to datetime
managers_df["start"] = pd.to_datetime(managers_df["start"], errors="coerce")
managers_df["end"] = pd.to_datetime(managers_df["end"], errors="coerce")

# Replace the table in the database
managers_df = replace_data("premier_league_managers", managers_df)
managers_df

Updated 477 rows in the premier_league_managers table


,manager,club,start,end,duration_days,years,incumbent,caretaker,incumbent_not_in_league
0,George Graham,Arsenal,1986-05-14,1995-02-21,3205,1992–1995,False,False,False
1,Stewart Houston,Arsenal,1995-02-22,1995-06-08,106,1995,False,True,False
2,Bruce Rioch,Arsenal,1995-06-08,1996-08-12,431,1995–1996,False,False,False
3,Stewart Houston,Arsenal,1996-08-12,1996-09-13,32,1996,False,True,False
4,Pat Rice,Arsenal,1996-09-13,1996-09-30,17,1996,False,True,False
...,...,...,...,...,...,...,...,...,...
472,Nuno Espírito Santo,Wolverhampton Wanderers,2017-05-31,2021-05-23,1453,2018–2021,False,False,False
473,Bruno Lage,Wolverhampton Wanderers,2021-06-09,2022-10-02,1149,2021–2022,False,False,False
474,Steve Davis,Wolverhampton Wanderers,2022-10-03,2022-11-13,41,2022,False,True,False
475,Julen Lopetegui,Wolverhampton Wanderers,2022-11-14,2023-08-08,267,2022–2023,False,False,False
